In [1]:
# Note: If an error that a lib is not defined although it is, restart the kernel

In [2]:
#from cookiecutter.main import cookiecutter
#cookiecutter("https://github.com/drivendata/cookiecutter-data-science")

In [3]:
#pip install -U threadpoolctl

# Intall this to resolve SMOTE AttributeError: 'NoneType' object has no attribute 'split'
#https://stackoverflow.com/questions/72297968/attributeerror-nonetype-object-has-no-attribute-split-smote

In [4]:
#@title codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MAS FS Pipeline

Mounted at /content/drive
/content/drive/MyDrive/MAS FS Pipeline


In [ ]:
# !pip install python-dotenv>=0.5.1
# !pip install scikit-learn==1.2.2
# !pip install mlforecast==0.9.2
# !pip install pandas==2.0.3
# !pip install matplotlib==3.7.3
# !pip install wordcloud==1.9.2
# !pip install tensorflow==2.10.1
# !pip install opt_einsum==3.3.0
# !pip install gast==0.5.4
# !pip install astunparse==1.6.3
# !pip install h5py==3.9.0
# !pip install future==0.18.3
# !pip install openpyxl==3.1.2
# !pip install torch==2.0.1
# !pip install torchvision==0.15.2
# !pip install torchaudio==2.0.2

!pip install sage-importance
!pip install shap==0.42.1
!pip install boruta
!pip install shap==0.42.1
!pip install imblearn==0.11.0
!pip install xgboost==2.0.0
!pip install numpy==1.23.5
!pip install torchmetrics==1.2.0
!pip install lifelines==0.27.8
!pip install joblib

In [6]:
#from feature_selection_timeseries.src.preprocessing.preprocessor import Preprocess
#from feature_selection_timeseries.src.data.data_importer import importer
#from feature_selection_timeseries.src.models.test_model import computeTestScore
#from feature_selection_timeseries.src.visualization.visualize import word_cloud_freq, plot_ts
#import copy
#from collections import Counter
#import ast
#import joblib
#import openpyxl
#import torch
#import time
#import matplotlib.pyplot as plt
#import os


from feature_selection_timeseries.src.models.pipeline import run
from feature_selection_timeseries.src.models.utils import create_time_feature, tune_cv_split, convert_to_sample #, map_data, check_column_types, roll_window_split, generate_val_splits
from datetime import datetime
import numpy as np
import pandas as pd
import warnings
import csv

warnings.filterwarnings("ignore", category=FutureWarning)

### Create Sample Data (Filter for data with the date specified)

In [7]:
year = "2006"
date_threshold = year + '-01-01'  # filter for data with date >=
sub_folder = "sp500_subset"

# path of the file containing the features
x_filename_in = 'stock_x.csv'
x_filename_out = f'stock_x_sample_regression_{year}_filtered_11_stocks.csv'
x_in_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/'
x_out_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/stocks/'

# path of the file containing the label
y_filename_in = 'stock_y_ret84.csv'
y_filename_out = f'stock_y_ret84_sample_regression_{year}_filtered_11_stocks.csv'
y_in_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/'
y_out_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/stocks/'

### Filter Data and Generate New Files

#### Run the codes below if you want to filter the original dataset for a selected subset of stocks

In [8]:
# stocks=[
#     "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"
# ]

# convert_to_sample(
#     path_in=x_in_path,
#     path_out=x_out_path,
#     filename_in = x_filename_in,
#     filename_out = x_filename_out,
#     date_threshold=date_threshold,
#     filter_type="combined stocks",
#     stocks=stocks
# )

# convert_to_sample(
#     path_in=y_in_path,
#     path_out=y_out_path,
#     filename_in = y_filename_in,
#     filename_out = y_filename_out,
#     date_threshold=date_threshold,
#     filter_type="combined stocks",
#     stocks=stocks
# )

### Import Sample Data from the Files with the Generated Subset of Features

In [10]:
# Specify the path to your CSV file
x_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/stock_x_sample_regression_2006_filtered_11_stocks.csv'
y_path = f'./feature_selection_timeseries/data/raw/{sub_folder}/stock_y_ret84_sample_regression_2006_filtered_11_stocks.csv'
# Import data
y = pd.read_csv(y_path)
sp500_df = pd.read_csv(x_path)
# Adjust labels
y['target'] = y['ret_fwd_84']
# Create additional time features
sp500_df = create_time_feature(sp500_df)
# Combine features and target
sp500_df = pd.concat([sp500_df.iloc[:, 1:], y['target']], axis=1)

In [11]:
# sp500_df = sp500_df[sp500_df['ticker'] == 'ADBE']
# y = y[y['ticker'] == 'ADBE']

In [12]:
# sp500_df = sp500_df.iloc[:1000, -100:]
# y = y.iloc[:1000, -100:]

In [15]:
display(y.head())
print(f"Dataset Shape: {np.shape(y)}")

,date,ticker,ret_fwd_84,target
0,2006-02-22,ADBE,-0.205624,-0.205624
1,2006-02-22,AMGN,-0.134793,-0.134793
2,2006-02-22,AZO,-0.052116,-0.052116
3,2006-02-22,BA,0.133960,0.133960
4,2006-02-22,CAT,-0.007569,-0.007569


Dataset Shape: (45616, 4)


In [16]:
display(sp500_df.head())
print(f"Dataset Shape: {np.shape(sp500_df)}")

,ticker,sector,RELSTR_M1,HIGHLOW_M1,STDDEV_M1,BOLL40_UP,BOLL40_LOW,BOLL40_SQUEEZE,BOLL40,BOLL60_UP,...,daaa,dbaa,dayofmonth,dayofweek,quarter,month,year,dayofyear,weekofyear,target
0,ADBE,2,0.009901,0.075963,0.019106,0.041396,-0.035230,0.076625,0.003083,0.041396,...,5.39,6.31,22,2,1,2,2006,53,8,-0.205624
1,AMGN,6,-0.049364,0.110197,0.031650,0.016843,-0.065948,0.082791,-0.024552,0.016843,...,5.39,6.31,22,2,1,2,2006,53,8,-0.134793
2,AZO,1,0.070841,0.084036,0.019217,0.015427,-0.042636,0.058063,-0.013605,0.015427,...,5.39,6.31,22,2,1,2,2006,53,8,-0.052116
3,BA,4,0.086246,0.123204,0.029887,0.009435,-0.097770,0.107204,-0.044167,0.009435,...,5.39,6.31,22,2,1,2,2006,53,8,0.133960
4,CAT,4,0.191753,0.201976,0.054908,0.012585,-0.127226,0.139812,-0.057321,0.012585,...,5.39,6.31,22,2,1,2,2006,53,8,-0.007569


Dataset Shape: (45616, 408)


### Initialize Pipeline

In [17]:
# Possible cross-validation splits
train_test_list = [tune_cv_split(
    sp500_df.iloc[-np.shape(sp500_df)[0]:,:],
    val_test_prop_constraint = 0.2, # Size of validation set relative to the train set
    num_split_constraint = 5 # Number of validation splits
)[-1]]

keep_data_index = train_test_list[0][0]*train_test_list[0][2] + 2*train_test_list[0][1]
print(f"\nUsing CV Split: {train_test_list}")

[[250, 50, 182], [500, 100, 90], [750, 150, 60], [1000, 200, 45], [1250, 250, 36], [1500, 300, 30], [1750, 350, 25], [2000, 400, 22], [2250, 450, 19], [2500, 500, 17], [2750, 550, 16], [3000, 600, 14], [3250, 650, 13], [3500, 700, 12], [3750, 750, 11], [4000, 800, 11], [4250, 850, 10], [4500, 900, 9], [4750, 950, 9], [5000, 1000, 8], [5250, 1050, 8], [5500, 1100, 7], [5750, 1150, 7], [6000, 1200, 7], [6250, 1250, 6], [6500, 1300, 6], [6750, 1350, 6], [7000, 1400, 6], [7250, 1450, 5], [7500, 1500, 5], [7750, 1550, 5], [8000, 1600, 5], [8250, 1650, 5]]

Using CV Split: [[8250, 1650, 5]]


In [18]:
r1 = run(
    cross_validation_type= "moving window", # or "expanding window"
    save_output_file = True, # Whether to save test outputs
    raw_df = sp500_df.iloc[-keep_data_index:, :].reset_index(drop=True), # Discard extra data instances from the beginning of the time series rather than the end
    y = y.iloc[-keep_data_index:, :].reset_index(drop=True), # Discard extra data instances from the beginning of the time series rather than the end
    train_test_list = train_test_list, # A list of possible list of train and validation size, and number of splits
    methods = ["xgboost", "cart"], # Available methods: ["xgboost", "cae", "permutation", "shap", "boruta", "sage", "lasso", "cart", "svm", "rf", "stg", "dynamic"]
    rebalance_type = ["None"], # ["borderlinesmote", "smoten", "random_over_sampler", "smote", "None"]
    label_cols = [], # Columns to label encode
    do_not_encode_cols = ["dayofmonth", "dayofweek", "quarter", "month", "year", "dayofyear", "weekofyear"], # These fields are not transformed
    seed = 42,
    target_colname = "target", # The name of the field that holds the true values
    dataset_name = "sp500",
    pred_type = "regression",
    append_to_full_df = False,
    n_features = 50,  # The number of top features to filter for
    feature_direction = "top", # Feature order based on their scores in descending order
    train_outputs_file_name = None,
    current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
    scaler_filename = "./feature_selection_timeseries/data/processed/scaler_saved.save",
    encoder_filename = "./feature_selection_timeseries/data/processed/encoder_saved.save",
    label_encoder_filename = "./feature_selection_timeseries/data/processed/lalbel_encoder_saved.save",
    test_output_file_name = f"./feature_selection_timeseries/data/final/Consolidated_Stocks_FS_timeseries_sp500_outputs_test_results_",
    test_pred_file_name = f"./feature_selection_timeseries/data/final/Consolidated_Stocks_FS_timeseries_sp500_outputs_test_preds_",
    print_outputs_train = False,
    print_outputs_test = True
)

#### Train model

In [ ]:
r1.train()

#### Test on holdout test data and generate testing outputs

In [ ]:
r1.test()